<a href="https://colab.research.google.com/github/soodeh-nilforoushan/NLP-Transformers/blob/main/Pipeline_Masked_Language_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Masked Language Model**
These models used by BERT, the models predict by both past and futures

Article spining idea: change enough words(while keeping the article coherent) such that it does not match the original text. Article spinning is really used in real world.

The MASK in our language model is a corrupted noise

In [1]:
!wget -nc https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv

--2023-08-15 02:29:33--  https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv
Resolving lazyprogrammer.me (lazyprogrammer.me)... 104.21.23.210, 172.67.213.166, 2606:4700:3030::ac43:d5a6, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|104.21.23.210|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5085081 (4.8M) [text/csv]
Saving to: ‘bbc_text_cls.csv’

bbc_text_cls.csv    100%[===================>]   4.85M  1.74MB/s    in 2.8s    

2023-08-15 02:29:37 (1.74 MB/s) - ‘bbc_text_cls.csv’ saved [5085081/5085081]



In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.4 MB/s eta 0:00:00


In [25]:
from transformers import pipeline, set_seed
import textwrap
import numpy as np
import pandas as pd
from pprint import pprint

In [26]:
df=pd.read_csv("bbc_text_cls.csv")

In [27]:
df.head()

,text,labels
0,Ad sales boost Time Warner profit\n\nQuarterly...,business
1,Dollar gains on Greenspan speech\n\nThe dollar...,business
2,Yukos unit buyer faces loan claim\n\nThe owner...,business
3,High fuel prices hit BA's profits\n\nBritish A...,business
4,Pernod takeover talk lifts Domecq\n\nShares in...,business


In [28]:
labels=set(df['labels'])
labels

{'business', 'entertainment', 'politics', 'sport', 'tech'}

In [17]:
#pick a label
label='business'

In [29]:
#we wanna filter out any rows do not match business and grab column containing text
#df['labels']==label ---> it return false or true
#df[df['labels']==label]['labels'] --> it grab rows with busines lable
texts=df[df['labels']==label]['text']
texts.head()

0    Ad sales boost Time Warner profit\n\nQuarterly...
1    Dollar gains on Greenspan speech\n\nThe dollar...
2    Yukos unit buyer faces loan claim\n\nThe owner...
3    High fuel prices hit BA's profits\n\nBritish A...
4    Pernod takeover talk lifts Domecq\n\nShares in...
Name: text, dtype: object

In [30]:
np.random.seed(1234)

In [46]:
#shape shows the number of rows and columns of a dataframe. shape[0]-->show the number of rows
#iloc[5] ---> shows the entire row number 5
i=np.random.choice(texts.shape[0])
doc=texts.iloc[i]


In [47]:
#print chosen article to see what it looks like.
print(textwrap.fill(doc, replace_whitespace=False, fix_sentence_endings=True))

Bombardier chief to leave company

Shares in train and plane-making
giant Bombardier have fallen to a 10-year low following the departure
of its chief executive and two members of the board.

Paul Tellier,
who was also Bombardier's president, left the company amid an ongoing
restructuring.  Laurent Beaudoin, part of the family that controls the
Montreal-based firm, will take on the role of CEO under a newly
created management structure.  Analysts said the resignations seem to
have stemmed from a boardroom dispute.  Under Mr Tellier's tenure at
the company, which began in January 2003, plans to cut the worldwide
workforce of 75,000 by almost a third by 2006 were announced.  The
firm's snowmobile division and defence services unit were also sold
and Bombardier started the development of a new aircraft seating 110
to 135 passengers.

Mr Tellier had indicated he wanted to stay at the
world's top train maker and third largest manufacturer of civil
aircraft until the restructuring was comple

In [49]:
mlm=pipeline('fill-mask')
#deafult model is distill Roberta which is derivative of bert

No model was supplied, defaulted to distilroberta-base and revision ec58a5b (https://huggingface.co/distilroberta-base).
Using a pipeline without specifying a model name and revision in production is not recommended.


Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [51]:
#test the model passing in the title of our chose article.
mlm('Bombardier chief to leave <mask>')
#these are the suggestion of the model, ex, Bombardier locates in Canada

[{'score': 0.06950785219669342,
  'token': 633,
  'token_str': ' job',
  'sequence': 'Bombardier chief to leave job'},
 {'score': 0.0669308453798294,
  'token': 1470,
  'token_str': ' France',
  'sequence': 'Bombardier chief to leave France'},
 {'score': 0.05273520573973656,
  'token': 558,
  'token_str': ' office',
  'sequence': 'Bombardier chief to leave office'},
 {'score': 0.025823047384619713,
  'token': 2201,
  'token_str': ' Paris',
  'sequence': 'Bombardier chief to leave Paris'},
 {'score': 0.021368566900491714,
  'token': 896,
  'token_str': ' Canada',
  'sequence': 'Bombardier chief to leave Canada'}]

In [54]:
#try the model again but with the next sentence
txt="Shares in <mask> and plane-making"+ \
"giant Bombardier have fallen to a 10-year low following the departure" + \
"of its chief executive and two members of the board."
mlm(txt)

[{'score': 0.6206227540969849,
  'token': 11016,
  'token_str': ' Airbus',
  'sequence': 'Shares in Airbus and plane-makinggiant Bombardier have fallen to a 10-year low following the departureof its chief executive and two members of the board.'},
 {'score': 0.32225683331489563,
  'token': 6722,
  'token_str': ' Boeing',
  'sequence': 'Shares in Boeing and plane-makinggiant Bombardier have fallen to a 10-year low following the departureof its chief executive and two members of the board.'},
 {'score': 0.0072267139330506325,
  'token': 8537,
  'token_str': ' airlines',
  'sequence': 'Shares in airlines and plane-makinggiant Bombardier have fallen to a 10-year low following the departureof its chief executive and two members of the board.'},
 {'score': 0.005469558760523796,
  'token': 14833,
  'token_str': ' Renault',
  'sequence': 'Shares in Renault and plane-makinggiant Bombardier have fallen to a 10-year low following the departureof its chief executive and two members of the board.'}

In [55]:
#try to mask another word! with this interface it is only possible to mask one word at a time
txt="Shares in train and plane-making"+ \
"giant Bombardier have fallen to a 10-year low following the departure" + \
"of its chief <mask> and two members of the board."
pprint(mlm(txt))


[{'score': 0.9821828007698059,
  'sequence': 'Shares in train and plane-makinggiant Bombardier have fallen to '
              'a 10-year low following the departureof its chief executive and '
              'two members of the board.',
  'token': 1031,
  'token_str': ' executive'},
 {'score': 0.012368573807179928,
  'sequence': 'Shares in train and plane-makinggiant Bombardier have fallen to '
              'a 10-year low following the departureof its chief executives '
              'and two members of the board.',
  'token': 4585,
  'token_str': ' executives'},
 {'score': 0.0014676136197522283,
  'sequence': 'Shares in train and plane-makinggiant Bombardier have fallen to '
              'a 10-year low following the departureof its chief officer and '
              'two members of the board.',
  'token': 1036,
  'token_str': ' officer'},
 {'score': 0.0010925761889666319,
  'sequence': 'Shares in train and plane-makinggiant Bombardier have fallen to '
              'a 10-year low foll